In [2]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("join").getOrCreate()
#spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1) 

# Create the first dataframe
df1 = spark.createDataFrame([("A", 1), ("B", 2), ("C", 3)], ["letter", "number"])

# Create the second dataframe
df2 = spark.createDataFrame([("A", 4), ("B", 5), ("D", 6)], ["letter", "value"])

# Perform the inner join
inner_join = df1.join(df2, df1['letter'] == df2['letter'], "inner")
inner_join.explain()

# Perform the inner join
inner_join = df1.hint('broadcast').join(df2, df1['letter'] == df2['letter'], "inner")
inner_join.explain()

spark.conf.set("spark.sql.join.preferSortMergeJoin","false")

# Perform the inner join
inner_join = df1.join(df2, df1['letter'] == df2['letter'], "inner")
inner_join.explain()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/29 14:05:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortMergeJoin [letter#0], [letter#4], Inner
   :- Sort [letter#0 ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(letter#0, 200), ENSURE_REQUIREMENTS, [plan_id=21]
   :     +- Filter isnotnull(letter#0)
   :        +- Scan ExistingRDD[letter#0,number#1L]
   +- Sort [letter#4 ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(letter#4, 200), ENSURE_REQUIREMENTS, [plan_id=22]
         +- Filter isnotnull(letter#4)
            +- Scan ExistingRDD[letter#4,value#5L]


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [letter#0], [letter#4], Inner, BuildLeft, false
   :- BroadcastExchange HashedRelationBroadcastMode(List(input[0, string, false]),false), [plan_id=46]
   :  +- Filter isnotnull(letter#0)
   :     +- Scan ExistingRDD[letter#0,number#1L]
   +- Filter isnotnull(letter#4)
      +- Scan ExistingRDD[letter#4,value#5L]


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false

In [3]:
# First dataframe
l = [(1, 0, 1, 1), (1, 0, 1, 1), (1, 0, 1, 1), (0, 0, 1, 1), ('', 0, 1, 1), ('', 0, 1, 1), (3, 0, 1, 1)]
rdd = spark.sparkContext.parallelize(l)
schema1 = [('num1')]
df = rdd.toDF(schema1)
df = df.drop('_2', '_3', '_4')
df.show()

# Second dataframe
l2 = [(1, 0, 1, 1), (1, 0, 1, 1), (0, 0, 1, 1), (0, 0, 1, 1), ('', 0, 1, 1), (4, 0, 1, 1)]
rdd2 = spark.sparkContext.parallelize(l2)
schema2 = [('num2')]
df2 = rdd2.toDF(schema2)
df2 = df2.drop('_2', '_3', '_4')
df2.show()

df.createOrReplaceTempView('df1')
df2.createOrReplaceTempView('df2')

# d1.num as num1, d2.num as num2

+----+
|num1|
+----+
|   1|
|   1|
|   1|
|   0|
|NULL|
|NULL|
|   3|
+----+

+----+
|num2|
+----+
|   1|
|   1|
|   0|
|   0|
|NULL|
|   4|
+----+



In [6]:
spark.sql("""
select num1, lag(num1, 2) over (order by num1) as lead_1
from df1
""").show()          


+----+------+
|num1|lead_1|
+----+------+
|NULL|  NULL|
|NULL|  NULL|
|   0|  NULL|
|   1|  NULL|
|   1|     0|
|   1|     1|
|   3|     1|
+----+------+



24/04/29 14:06:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/29 14:06:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/29 14:06:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/29 14:06:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/29 14:06:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
